In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
linkTrain = 'https://drive.google.com/file/d/1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI/view?usp=sharing'
linkIdTrain = '1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI'

linkTest = 'https://drive.google.com/file/d/1QIJWtK99SblVQK88AHqHwAVBZJcT14yW/view?usp=sharing'
LinkIdTest = '1QIJWtK99SblVQK88AHqHwAVBZJcT14yW'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
downloaded = drive.CreateFile({'id':linkIdTrain}) 
downloaded.GetContentFile('Bot_IOT_Train.csv')  

downloaded = drive.CreateFile({'id':LinkIdTest}) 
downloaded.GetContentFile('Bot_IOT_Test.csv')  

#loading data
train_df = pd.read_csv('Bot_IOT_Train.csv')
test_df = pd.read_csv('Bot_IOT_Test.csv')

In [ ]:
test_df.head()

,pkSeqID,proto,saddr,sport,daddr,dport,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack,category,subcategory
0,792371,udp,192.168.100.150,48516,192.168.100.3,80,175094,0.226784,100,4.100436,4,4.457383,100,0.000000,0.404711,4.719438,1,DoS,UDP
1,2056418,tcp,192.168.100.148,22267,192.168.100.3,80,143024,0.451998,100,3.439257,1,3.806172,100,0.225077,0.401397,4.442930,1,DDoS,TCP
2,2795650,udp,192.168.100.149,28629,192.168.100.3,80,167033,1.931553,73,0.000000,4,2.731204,100,0.000000,0.407287,4.138455,1,DDoS,UDP
3,2118009,tcp,192.168.100.148,42142,192.168.100.3,80,204615,0.428798,56,3.271411,1,3.626428,100,0.000000,0.343654,4.229700,1,DDoS,TCP
4,303688,tcp,192.168.100.149,1645,192.168.100.5,80,40058,2.058381,100,0.000000,3,1.188407,100,0.000000,0.135842,4.753628,1,DoS,TCP


In [ ]:
print(train_df.shape)
print(test_df.shape)

(2934817, 19)
(733705, 19)


In [ ]:
train_df.drop(["pkSeqID","proto","saddr", "sport", "daddr", "dport"], axis=1, inplace=True)
test_df.drop(["pkSeqID","proto","saddr", "sport", "daddr", "dport"], axis=1, inplace=True)

In [ ]:
train_df.dtypes

seq                    int64
stddev               float64
N_IN_Conn_P_SrcIP      int64
min                  float64
state_number           int64
mean                 float64
N_IN_Conn_P_DstIP      int64
drate                float64
srate                float64
max                  float64
attack                 int64
category              object
subcategory           object
dtype: object

In [ ]:
train_df.isnull().sum()

seq                  0
stddev               0
N_IN_Conn_P_SrcIP    0
min                  0
state_number         0
mean                 0
N_IN_Conn_P_DstIP    0
drate                0
srate                0
max                  0
attack               0
category             0
subcategory          0
dtype: int64

In [ ]:
unique_count=train_df.nunique()
unique_count

seq                  262206
stddev               401054
N_IN_Conn_P_SrcIP       100
min                  256015
state_number             11
mean                 479462
N_IN_Conn_P_DstIP       100
drate                 18952
srate                114098
max                  558914
attack                    2
category                  5
subcategory               8
dtype: int64

In [ ]:
Y=train_df.loc[:,'attack']
y=test_df.loc[:,'attack']

In [ ]:
X=train_df.iloc[:, 0:10]
x=test_df.iloc[:,0:10]

In [ ]:
print(train_df.shape)
print(test_df.shape)

(2934817, 13)
(733705, 13)


In [ ]:
Y.value_counts()

1    2934447
0        370
Name: attack, dtype: int64

In [ ]:
y.value_counts()

1    733598
0       107
Name: attack, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split as tts
X_Train, X_Test, Y_Train, Y_Test = tts(X,Y, test_size = 0.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#creating and training classifier
model = RandomForestClassifier(n_estimators=100, criterion="entropy")
model.fit(X_Train, Y_Train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#predicting VALIDATION SET
y_val = model.predict(X_Test)
y_val

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
#model evaluation over VALIDATION
accuracy = accuracy_score(Y_Test,y_val)*100
accuracy

99.99897779080148

In [ ]:
#confusion matrix VALIDATION
print("Confusion Matrix:")
print(confusion_matrix(Y_Test, y_val))

Confusion Matrix:
[[   103      3]
 [     6 880334]]


In [ ]:
print("Classification Report")
print(classification_report(Y_Test, y_val))

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       106
           1       1.00      1.00      1.00    880340

    accuracy                           1.00    880446
   macro avg       0.97      0.99      0.98    880446
weighted avg       1.00      1.00      1.00    880446



In [ ]:
# for TEST SET
y_pred = model.predict(x)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
#model evaluation over TEST SET
accuracy = accuracy_score(y,y_pred)*100
accuracy

99.99890964352159

In [ ]:
#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))

Confusion Matrix:
[[   101      6]
 [     2 733596]]


In [ ]:
print("Classification Report")
print(classification_report(y, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       107
           1       1.00      1.00      1.00    733598

    accuracy                           1.00    733705
   macro avg       0.99      0.97      0.98    733705
weighted avg       1.00      1.00      1.00    733705



In [ ]:
#encoding category and subcategory
#encoding 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
train_df['category']=le.fit_transform(train_df['category'])
train_df['subcategory']=le1.fit_transform(train_df['subcategory'])

test_df['category']=le.transform(test_df['category'])
test_df['subcategory']=le1.transform(test_df['subcategory'])

In [ ]:
#after attack, predict type
X_TrainCat = train_df.iloc[:, 0:11]
Y_TrainCat = train_df.loc[:, 'category']

X_TestCat = test_df.iloc[:,0:10]
X_TestCat['attack'] = y_pred
Y_TestCat = test_df.loc[: , 'category']

Y_TestCat
Y_TrainCat = Y_TrainCat.astype('int') 

In [ ]:
#creating and training classifier
model1 = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=2)
model1.fit(X_TrainCat, Y_TrainCat)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#calcuating accuracy of the model on test set
pred_cat = model1.predict(X_TestCat)

In [ ]:
#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_TestCat, pred_cat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_TestCat, pred_cat))

#classification report
print("Classification Report")
print(classification_report(Y_TestCat, pred_cat))

Accuracy Score: 
81.26944753000184
Confusion Matrix:
[[358663  26646      0      0      0]
 [ 92497 237615      0      0      0]
 [     8     99      0      0      0]
 [ 10079   8084      0      0      0]
 [     5      9      0      0      0]]
Classification Report


C:\Users\B.K Goel\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.78      0.93      0.85    385309
           1       0.87      0.72      0.79    330112
           2       0.00      0.00      0.00       107
           3       0.00      0.00      0.00     18163
           4       0.00      0.00      0.00        14

    accuracy                           0.81    733705
   macro avg       0.33      0.33      0.33    733705
weighted avg       0.80      0.81      0.80    733705



In [ ]:
####################Predicting subcategory of attack
#dependent and independent variables
X_subcat_train = train_df.iloc[:, 0:12]
Y_subcat_train = train_df.loc[:, 'subcategory']

X_subcat_test =test_df.iloc[:, 0:10]

X_subcat_test['attack']= y_pred
X_subcat_test['category']= pred_cat
Y_subcat_test=test_df.loc[:, 'subcategory']


In [ ]:
#classifying
model_rf_subcat = RandomForestClassifier(n_estimators=100, criterion = "entropy")
model_rf_subcat.fit(X_subcat_train, Y_subcat_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#calcuating accuracy of the model on test set
pred_subcat = model_rf_subcat.predict(X_subcat_test)

#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_subcat_test, pred_subcat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_subcat_test, pred_subcat))

Accuracy Score: 
98.1612500937025
Confusion Matrix:
[[   502      0      0      0      0      2      0]
 [     9      4      0      0      0      0      1]
 [    11      0     50      0      0     36     10]
 [   262      0      0    774     13   2553     19]
 [  1590      0      0      6   4016   8884     46]
 [    41      0      0      0      0 318294      2]
 [     0      0      0      0      0      6 396574]]


In [ ]:
#df['attack', 'category', 'subcategory']
a=test_df['attack']
b=test_df['category']
c=test_df['subcategory']
count=0
for i in range(0,len(test_df)):
  if(y_pred[i]==a[i]) and (pred_cat[i]==b[i]) and (pred_subcat[i]==c[i]):
    count=count+1
count
count/len(test_df)*100

81.26903864632243